In [1]:
import tensorflow._api.v2.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np
import pickle 

tf.disable_v2_behavior()
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)

WILDCARD_CHAR = '\t'
MAX_LEN = 16
CHARMAP_PATH = '../DATA/AE_char_map.pickle'
MODEL_PATH = '../DATA/AE_based_Noise_a4_ls128_hn4_al8/'

2023-11-05 22:06:27.613233: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
templates = ['p***r', 'j**n*', 'd*m***c']
templates = list(map(lambda t: t.replace('*', WILDCARD_CHAR), templates))
templates

['p\t\t\tr', 'j\t\tn\t', 'd\tm\t\t\tc']

In [3]:
with open(CHARMAP_PATH, 'rb') as f:
    char2id: dict[str, int] = pickle.load(f)

id2char = [''] * len(char2id)
for char, id in char2id.items():
    id2char[id] = char

In [4]:

# id2char

In [5]:
module = hub.Module(MODEL_PATH)
x_placeholder = tf.placeholder(tf.int32, shape=(None, MAX_LEN))

# number of samples
n = 10000
# standard devation
stddev = 0.6

module_inputs = {'x': x_placeholder, 'stddev': (stddev,), 'n': (n,)}
module_dict = module(module_inputs, as_dict=True, signature='sample_from_latent')
prediction_tensor = module_dict['prediction']
module_dict

{'logits': <tf.Tensor 'module_apply_sample_from_latent/dec_2/conv1d_18/BiasAdd:0' shape=(10000, 16, 213) dtype=float32>,
 'prediction_string': <tf.Tensor 'module_apply_sample_from_latent/ReduceJoin_2:0' shape=(?,) dtype=string>,
 'p': <tf.Tensor 'module_apply_sample_from_latent/Softmax_2:0' shape=(10000, 16, 213) dtype=float32>,
 'prediction': <tf.Tensor 'module_apply_sample_from_latent/ArgMax_4:0' shape=(10000, 16) dtype=int32>,
 'z': <tf.Tensor 'module_apply_sample_from_latent/enc_1/dense/BiasAdd:0' shape=(1, 128) dtype=float32>,
 'default': <tf.Tensor 'module_apply_sample_from_latent/Softmax_2:0' shape=(10000, 16, 213) dtype=float32>}

In [ ]:
produced_strings = []

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    for template in templates:
        template_as_vector = np.zeros(MAX_LEN, dtype=np.int32)
        for i, c in enumerate(template): 
            template_as_vector[i] = char2id.get(c, -1)
        template_as_vector = template_as_vector[None, :]

        samples = sess.run(prediction_tensor, {x_placeholder: template_as_vector})

        samples_as_strings = list(map(
            lambda vec: ''.join(id2char[i] for i in vec if i > 0),
            samples
        ))

        samples_filtered = list(set(filter(
            lambda sample: len(sample) == len(template)
                and all(sc == tc or tc == WILDCARD_CHAR
                        for sc, tc in zip(sample, template)),
            samples_as_strings
        )))

        produced_strings.extend(samples_filtered)

produced_strings

In [7]:
len(produced_strings)

1111

In [8]:
'dominic' in produced_strings

True

In [9]:
'peter' in produced_strings

True

In [10]:
'john' in produced_strings

False